In [1]:
import os
path = "D:\projects\zimpleton\\backend\\app"
os.chdir(path)

In [2]:
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import columns
from cassandra.cqlengine import connection
from cassandra.cqlengine.management import sync_table


from lib.connections import *
from lib.models import *
from lib.schemas import *
from lib.helpers import *
from lib.utils import *

INFO : CONNECTING TO CASSANDRA
INFO : SUCCESS CONNECTION TO CASSANDRA
INFO : REDIS CONNECTION NOT SETUP


In [14]:
query = "SELECT keyspace_name FROM system_schema.keyspaces;"
ckeyspaces = connection.execute(query)
ckeyspaces = [keyspace["keyspace_name"] for keyspace in ckeyspaces]
ckeyspaces

['xkeyspace',
 'unispace',
 'system_auth',
 'system_schema',
 'system',
 'datastax_sla',
 'pinspace',
 'system_traces',
 'data_endpoint_auth']

In [26]:
connection.execute("truncate table user_relation")
# connection.execute("drop table user_relation")

In [27]:
class UserRelation(Model):
    follower_email = columns.Text(primary_key = True)
    followee_email = columns.Text(primary_key=True)
    
    follower_name = columns.Text()
    followee_name = columns.Text()

sync_table(UserRelation)

In [15]:
relations = UserRelation.objects.all()


[UserRelation(follower_email='michael@gmail.com', followee_email='jack@gmail.com', follower_name='michael', followee_name='jack'),
 UserRelation(follower_email='michael@gmail.com', followee_email='jordan@gmail.com', follower_name='michael', followee_name='jordan'),
 UserRelation(follower_email='michael@gmail.com', followee_email='mickey@gmail.com', follower_name='michael', followee_name='mickey'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='alex@gmail.com', follower_name='mickey', followee_name='alex'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='jordan@gmail.com', follower_name='mickey', followee_name='jordan'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='michael@gmail.com', follower_name='mickey', followee_name='michael'),
 UserRelation(follower_email='jack@gmail.com', followee_email='jordan@gmail.com', follower_name='jack', followee_name='jordan'),
 UserRelation(follower_email='jack@gmail.com', followee_email='michael@gm

In [6]:
names = ['jack', 'mickey', 'michael', 'jordan']
mock_data = [
    {"follower_email": f"{name}@gmail.com", "followee_email": f"{name2}@gmail.com", "follower_name": name, "followee_name": name2} # added follower_name here
    for name in names for name2 in names if name != name2
]

In [7]:
from cassandra.cqlengine.query import BatchQuery

# values = UserRelation.objects.all()
# list(values)
# user_relation = UserRelation(follower_email = "jake@gmail.com", follower_name = "jake" , followee_email = "emily@gmail.com", followee_name = "emily")

for value in mock_data:
    UserRelation.create(**value)



In [8]:
values = UserRelation.objects.all();
list(values)

[UserRelation(follower_email='michael@gmail.com', followee_email='jack@gmail.com', follower_name='michael', followee_name='jack'),
 UserRelation(follower_email='michael@gmail.com', followee_email='jordan@gmail.com', follower_name='michael', followee_name='jordan'),
 UserRelation(follower_email='michael@gmail.com', followee_email='mickey@gmail.com', follower_name='michael', followee_name='mickey'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='alex@gmail.com', follower_name='mickey', followee_name='alex'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='jack@gmail.com', follower_name='mickey', followee_name='jack'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='jordan@gmail.com', follower_name='mickey', followee_name='jordan'),
 UserRelation(follower_email='mickey@gmail.com', followee_email='michael@gmail.com', follower_name='mickey', followee_name='michael'),
 UserRelation(follower_email='jack@gmail.com', followee_email='jordan@gma

In [9]:
def get_followers(followee_email):
    followers = UserRelation.objects.allow_filtering().filter(followee_email=followee_email)

    if not followers:
        return []

    followers = [{'name': relation.follower_name, 'email': relation.follower_email} for relation in followers]
    return followers

def get_following(follower_email):
    following = UserRelation.objects.allow_filtering().filter(follower_email=follower_email)

    if not following:
        return []

    following = [{'name': relation.followee_name, 'email': relation.followee_email} for relation in following]
    return following


def follow(follower_email, follower_name, followee_email, followee_name ):
    try:
        UserRelation.create(
            followee_email=followee_email,
            follower_email=follower_email,
            followee_name=followee_name,
            follower_name=follower_name
        )
    except Exception as e:
        print("INFO : UserRelation Creation Error")
        print(e)
        #raise exception here

def unfollow(followee_email, follower_email):
    try:
        relation = UserRelation.get(followee_email=followee_email, follower_email=follower_email)
        relation.delete()
    except:
        pass

In [45]:
user_email = "doesnotexit@gmail.com"
user_email = 'ip@gmail.com'

user_name = UserInfo.get(email = user_email).name
user_name

'ip@gmail.com'

In [57]:
email = 'pi@gmail.com'
get_followers(followee_email= email)

[{'name': 'steve@gmail.com', 'email': 'steve'},
 {'name': 'micheal@gmail.com', 'email': 'micheal'},
 {'name': 'mickey@gmail.com', 'email': 'mickey'},
 {'name': 'jimmy@gmail.com', 'email': 'jimmy'},
 {'name': 'alex@gmail.com', 'email': 'alex'},
 {'name': 'john@gmail.com', 'email': 'john'}]

In [56]:
get_following(follower_email='ip@gmail.com')

[{'name': 'alex', 'email': 'alex@gmail.com'},
 {'name': 'jimmy', 'email': 'jimmy@gmail.com'},
 {'name': 'john', 'email': 'john@gmail.com'},
 {'name': 'micheal', 'email': 'micheal@gmail.com'},
 {'name': 'mickey', 'email': 'mickey@gmail.com'},
 {'name': 'steve', 'email': 'steve@gmail.com'}]

In [33]:
follower_email = follower_name = 'ip@gmail.com'

followee_names = ["mickey", "micheal", "jimmy", "john", "alex", "steve"]

get_followee_email = lambda name: f"{name}@gmail.com"
followee_emails = [get_followee_email(name) for name in followee_names]

# followee_email
# follower_email
# followee_name
# follower_name


for followee_email, followee_name in zip(followee_emails, followee_names):
    
    follow(
        followee_email = followee_email,
        follower_email = follower_email,
        followee_name = followee_name,
        follower_name = follower_name,
    )

In [54]:
followee_email = followee_name = 'pi@gmail.com'

follower_names = ["mickey", "micheal", "jimmy", "john", "alex", "steve"]

get_follower_email = lambda name: f"{name}@gmail.com"
follower_emails = [get_follower_email(name) for name in followee_names]

# followee_email
# follower_email
# followee_name
# follower_name

for follower_email, follower_name in zip(follower_names, follower_emails):
    
    follow(
        followee_email = followee_email,
        follower_email = follower_email,
        followee_name = followee_name,
        follower_name = follower_name,
    )

In [12]:
unfollow(follower_email = 'ip@gmail.com', followee_email='jack@gmail.com')

In [13]:
followee_email = 'doesnotexist@gmail.com'
followee_email = 'ip@gmail.com'
followers = UserRelation.objects.allow_filtering().filter(followee_email=followee_email)

print(list(followers))

[{'name': 'alex', 'email': 'alex@gmail.com'},
 {'name': 'jordan', 'email': 'jordan@gmail.com'},
 {'name': 'michael', 'email': 'michael@gmail.com'}]

In [61]:
values = UserCredentials.objects.all()
values = UserInfo.objects.all()
list(values)

[UserInfo(email='pi@gmail.com', name='pi@gmail.com'),
 UserInfo(email='ip@gmail.com', name='ip@gmail.com')]